# Filtros em Componentes

Alguns componentes do notebook anterior:

## DataSet

In [1]:
public interface ITableProducer {
  String[] requestAttributes();
  String[][] requestInstances();
}

com.twosigma.beaker.javash.bkrcfbe28a4.ITableProducer

In [2]:
public interface IDataSetProperties {
  public String getDataSource();
  public void setDataSource(String dataSource);
}

com.twosigma.beaker.javash.bkrcfbe28a4.IDataSource

In [3]:
public interface IDataSet extends IDataSetProperties, ITableProducer {
}

com.twosigma.beaker.javash.bkrcfbe28a4.IDataSet

In [4]:
import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.util.ArrayList;

public class DataSetComponent implements IDataSet {
  private String dataSource = null;
  private String[] attributes = null;
  private String[][] instances = null;
  
  public DataSetComponent() {
    /* nothing */
  }

  public String getDataSource() {
    return dataSource;
  }

  public void setDataSource(String dataSource) {
    this.dataSource = dataSource;
    if (dataSource == null) {
      attributes = null;
      instances = null;
    } else
      readDS();
  }
  
  public String[] requestAttributes() {
    return attributes;
  }
  
  public String[][] requestInstances() {
    return instances;
  }
  
  private void readDS() {
    ArrayList<String[]> instArray = new ArrayList<String[]>();
    try {
      BufferedReader file = new BufferedReader(new FileReader(dataSource));
        
      String line = file.readLine();
      if (line != null) {
        attributes = line.split(",");
        line = file.readLine();
        while (line != null) {
          String[] instLine = line.split(",");
          instArray.add(instLine);
          line = file.readLine();
        }
        instances = instArray.toArray(new String[0][]);
      }
        
      file.close();
    } catch (IOException erro) {
      erro.printStackTrace();
    }
  }
  
}

com.twosigma.beaker.javash.bkrcfbe28a4.DataSetComponent

## Console

In [5]:
public interface ITableProducerReceptacle {
  public void connect(ITableProducer producer);
}

com.twosigma.beaker.javash.bkrcfbe28a4.ITableProducerReceptacle

In [6]:
public interface IConsoleUpdate {
  public void update();
}

com.twosigma.beaker.javash.bkrcfbe28a4.IConsoleUpdate

In [7]:
public interface IConsole extends ITableProducerReceptacle, IConsoleUpdate {
}

com.twosigma.beaker.javash.bkrcfbe28a4.IConsole

In [8]:
public class ConsoleComponent implements IConsole {
  private ITableProducer iProducer;
  
  public void connect(ITableProducer producer) {
    iProducer = producer;
  }
  
  public void update() {
    if (iProducer != null) {
        System.out.println("=== Attributes ===");
        String attributes[] = iProducer.requestAttributes();
        for (int a = 0; a < attributes.length-1; a++)
          System.out.print(attributes[a] + ", ");
        System.out.println(attributes[attributes.length-1]);

        System.out.println();
        System.out.println("=== Instances ===");
        String instances[][] = iProducer.requestInstances();
        for (int i = 0; i < instances.length; i++) {
          for (int a = 0; a < attributes.length-1; a ++)
            System.out.print(instances[i][a] + ", ");
          System.out.println(instances[i][attributes.length-1]);
        }
    }
  }
}

com.twosigma.beaker.javash.bkrcfbe28a4.ConsoleComponent

# Projeção

In [9]:
public interface IProjectionProperties extends ITableProducer {
  String getAttribute();
  void setAttribute(String attribute);
}

com.twosigma.beaker.javash.bkrcfbe28a4.IProjectionServices

In [ ]:
public interface IProjection extends IProjectionServices, ITitle, ITableProducerReceptacle {
}

In [ ]:
import java.util.Enumeration;

public class ProjectionComponent implements IProjection {
  private ITableProducer provider;
  
  private String attribute = null,
                 title = null;
  
  public String getAttribute() {
    return attribute;
  }

  public void setAttribute(String attribute) {
    this.attribute = attribute;
  }
  
  public String getTitle() {
    String title = (this.title != null) ? this.title : attribute;
    return title;
  }
  
  public void setTitle(String title) {
    this.title = title;
  }
  
  public void connect(ITableProducer provider) {
    this.provider = provider;
  }
  
  public double[] requestValues() {
    double[] attributeValues = null;
    
    if (provider != null) {
      Instances instances = provider.requestInstancesWeka();
      
      Attribute attr = findAttribute(instances);
      if (attr != null) {
        int index = attr.index();
        if (index != -1)
          attributeValues = instances.attributeToDoubleArray(index);
      }
    }
    
    return attributeValues;
  }
  
  /* (non-Javadoc)
   * @see pt.c08componentes.s20catalog.s20projection.IProjectionSimple#projectAttributeNominals()
   */
  @Override
  public String[] requestNominals() {
    String[] nominals = null;
    Instances instances = provider.requestInstancesWeka();
    
    Attribute attr = findAttribute(instances);
    
    boolean isNominal = attr.isNominal();
    
    if (attr != null) {
      int index = attr.index();

      if (index != -1) {
        nominals = new String[instances.size()];
        for (int i = 0; i < nominals.length; i++)
          nominals[i] = (isNominal)?
                          instances.get(i).stringValue(index) :
                          Double.toString(instances.get(i).value(index));
      }
    }
    
    return nominals;
  }
  
  private Attribute findAttribute(Instances instances) {
    Attribute attrFinal = null;
    
    if (attribute != null) {
      Enumeration<Attribute> attrEnum = instances.enumerateAttributes();

      while (attrFinal == null && attrEnum.hasMoreElements()) {
        Attribute attr = attrEnum.nextElement();
        if (attribute.equalsIgnoreCase(attr.name()))
          attrFinal = attr;
      }
    }
    
    return attrFinal;
  }
  
  @Override
  public String toString() {
    String output = "***** empty *****";
    
    Instances instances = provider.requestInstancesWeka();
    Attribute attr = findAttribute(instances);
    
    if (attr != null) {
      output = "===== " + getTitle() + " =====\n";
      if (attr.isNominal()) {
        String[] values = requestNominals();
        if (values != null) {
          for (String val: values)
            output += val + "\n";
        }
      } else {
        double[] values = requestValues();
        if (values != null) {
          for (double val: values)
            output += val + "\n";
        }
      }
    }
    
    return output;
  }
}